In [1]:
import os
os.chdir('/home/roberto/Documents/Titulación/Tesina')

In [2]:
from scripts import *

### Países disponibles:

##### -España
##### -Alemania

### Años disponibles:

##### 2008 a 2017

Correr siguiente celda para repetir toda la simulación

In [3]:
#todo = {}
#for year in years:
#    for country in paises:
#        solar_year(country,year).to_pandas().to_csv('../Archivos/Output/'+country+str(year)+'.csv')
#        todo[country+str(year)]=pd.read_csv('../Archivos/Output/'+country+str(year)+'.csv')

Correr siguiente celda para leer archivos existentes de simulación anterior

In [4]:
import pandas as pd
os.chdir('/home/roberto/Documents/Titulación/Archivos/Output')
todo = {}
for year in years:
    for country in paises:
        todo[country+str(year)]=pd.read_csv(country+str(year)+'.csv')